In [6]:
# pip uninstall kafka-python

In [2]:
# pip install --upgrade pyspark

  Using cached pyspark-3.4.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.7
    Uninstalling py4j-0.10.7:
      Successfully uninstalled py4j-0.10.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 2.4.6
    Uninstalling pyspark-2.4.6:
      Successfully uninstalled pyspark-2.4.6
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import pyarrow.parquet as pq

# Initialize Spark session
spark = SparkSession.builder.appName('LogProcessing').getOrCreate()

# Initialize StreamingContext with batch interval of 10 seconds
ssc = StreamingContext(spark.sparkContext, 10)

# Set Kafka consumer parameters
kafka_params = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'log_consumer_group',
    'auto.offset.reset': 'earliest'
}
kafka_topics = ['log_topic']

# Create Kafka direct stream
dstream = KafkaUtils.createDirectStream(ssc, kafka_topics, kafka_params)

# Extract log data from Kafka messages
log_data = dstream.map(lambda x: x[1])  # Assuming messages are in (key, value) format, extract the value

# Perform transformations and EDA using Spark
transformed_data = log_data.map(lambda x: your_transformation_function(x))  # Apply your transformation function

# Convert DStream to DataFrame
transformed_data.foreachRDD(lambda rdd: rdd.toDF().createOrReplaceTempView('temp_table'))

# Write DataFrame to Parquet file
spark.sql('SELECT * FROM temp_table').write.parquet('/path/to/parquet_file')

# Store Parquet file in HDFS
hdfs_path = 'hdfs://localhost:9000/path/to/parquet_file'
local_path = '/path/to/parquet_file'
pq.write_to_dataset(table=spark.table('temp_table'), root_path=hdfs_path, compression='snappy')

# Start the streaming context
ssc.start()
ssc.awaitTermination()


ModuleNotFoundError: No module named 'pyspark.streaming.kafka'